In [1]:
import optuna

# Часть 1. Поиск минимума функции при помощи optuna

### Функция Розенброка

$$((1 - x) ^ 2 + 100 * (y - x * x) ^ 2 )$$

In [2]:
def f1(x,y):
    return ((1 - x) ** 2 + 100 * (y - x * x) ** 2 )
def objective1(trial):
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -10, 10)
    return f1(x,y)

### Функция Била 

$$(1.5 - x + x*y)^2 + (2.25 - x + x*y^2)^2 + (2.625 - x + x*y^3)^2$$

In [3]:
def f2(x,y):
    return (1.5 - x + x*y)**2 + (2.25 - x + x*y**2)**2 + (2.625 - x + x*y**3)**2
def objective2(trial):
    x = trial.suggest_float("x", -10, 10)
    y = trial.suggest_float("y", -4.5, 4.5)
    return f2(x,y)

In [4]:
optuna.logging.disable_default_handler() #Отключить логирование
study1 = optuna.create_study()
study1.optimize(objective1, n_trials=400)
study2 = optuna.create_study()
study2.optimize(objective2, n_trials=400)

In [5]:
best_params = study1.best_params
found_x = best_params["x"]
found_y = best_params["y"]
print("###### Результат на функции Розенброка ######")
print("Найденные    x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(found_x, found_y, f1(found_x,found_y)))
print("Оптимальные  x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(1, 1, f1(1,1)))
print("Разница      x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(abs(found_x-1), abs(found_y-1), abs(f1(found_x,found_y)-f1(1,1))))

###### Результат на функции Розенброка ######
Найденные    x: 0.797 y: 0.612, F(x,y): 0.092
Оптимальные  x: 1.000 y: 1.000, F(x,y): 0.000
Разница      x: 0.203 y: 0.388, F(x,y): 0.092


In [6]:
best_params = study2.best_params
found_x = best_params["x"]
found_y = best_params["y"]
print("####### Результат на функции Била #######")
print("Найденные    x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(found_x, found_y, f2(found_x,found_y)))
print("Оптимальные  x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(3, 0.5, f2(3,0.5)))
print("Разница      x: {:0.3f} y: {:0.3f}, F(x,y): {:0.3f}".format(abs(found_x-3), abs(found_y-0.5), abs(f2(found_x,found_y)-f2(3,0.5))))

####### Результат на функции Била #######
Найденные    x: 3.021 y: 0.508, F(x,y): 0.000
Оптимальные  x: 3.000 y: 0.500, F(x,y): 0.000
Разница      x: 0.021 y: 0.008, F(x,y): 0.000


# Часть 2. Решение новой задачи из scikit-learn моделью из scikit-learn, используя optuna для поиска гиперпараметров

In [7]:
import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import optuna

SEED = 0

## Набор данных wine - классификация на 3 класса с 13 признаками на 178 элемента выборки
## Модель KNeighborsClassifier - классификатор, реализующий голосование k ближайших соседей

In [8]:
def objective(trial):
    data, target = sklearn.datasets.load_wine(return_X_y=True) #Набор данных о вине (классификация на 3 класса).
    train_x, valid_x, train_y, valid_y = train_test_split(data, target, test_size=0.20)
    
    n_neighbors = trial.suggest_int("n_neighbors", 1, 10)
    algorithm = trial.suggest_categorical("algorithm", ["auto", "ball_tree", "kd_tree", "brute"])
    leaf_size = trial.suggest_int("leaf_size", 1, 100, log=True)
    
    neigh = KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=algorithm, leaf_size=leaf_size)
    neigh.fit(train_x, train_y)
    KNeighborsClassifier()
    
    return neigh.score(valid_x,valid_y)

In [11]:
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=SEED),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=100),
)
study.optimize(objective, n_trials=1000)

In [12]:
best_params = study.best_params
n_neighbors = best_params["n_neighbors"]
algorithm = best_params["algorithm"]
leaf_size = best_params["leaf_size"]
print("Лучшими score = {:0.3f}, достигнутый при параметрах: \nn_neighbors = {}, \nalgorithm = \"{}\", \nleaf_size = {}.".format(
      study.best_trial.values[0], n_neighbors, algorithm,leaf_size))

Лучшими score = 0.944, достигнутый при параметрах: 
n_neighbors = 1, 
algorithm = "brute", 
leaf_size = 18.
